In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"eos_token": "<|endoftext|>"})  # EOS zaten var
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer.bos_token, tokenizer.eos_token, tokenizer.pad_token)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

<|endoftext|> <|endoftext|> <|endoftext|>


In [ ]:
chr = "<|endoftext|> hello world from turkey"
tokenized = tokenizer(chr, truncation=True, max_length=32, padding='max_length', return_tensors="np")
print(tokenized)
print(tokenizer.bos_token_id)

{'input_ids': array([[50256, 23748,   995,   422, 26876,   257,    80, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}
50256


In [ ]:

import torch
from transformers import CLIPTokenizer, CLIPTextModel

# CLIP modelini yükleyelim


# Tokenizer ve text encoder'ı yükle

# Vocab boyutunu ve embedding boyutunu al

# CLIP'in Text Embedding Ağırlıklarını Al
print(tokenizer.eos_token)
print(tokenizer.bos_token)
print(tokenizer.pad_token)
print(tokenizer.vocab_size)
import json
import numpy as np
import re
vocab_size = tokenizer.vocab_size
# 📌 Sabitler
MAX_TOKENS = 32  # Maksimum token sayısı
FEATURE_DIM = 512  # CLIP özellik boyutu
MAX_SEGMENT_FRAMES = 15  # Her segment için maksimum 15 frame

# 📌 CLIP Özelliklerini Yükleme
def load_clip_features(clip_feature_paths):
    video_features_dict = {}  # Video ID → Özellikler
    for path in clip_feature_paths:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for video_id, features in data.items():
                if isinstance(features, dict):
                    features = np.array(list(features.values()), dtype=np.float32)
                    features = features[0]

                else:
                    features = np.array(features, dtype=np.float32)
                    features = features[0]
                # 📌 İlk boyut kontrolü
                if len(features.shape) != 2 or features.shape[1] != FEATURE_DIM:
                    print(f"⚠️ Uyarı: {video_id} için geçersiz özellik boyutu: {features.shape}")
                    continue

                video_features_dict[f"v_{video_id}"] = features  # ID formatı düzeltildi!
    return video_features_dict

# 📌 Verileri Hazırlama (Segment Bazlı Input)
def prepare_training_data(json_paths, video_features_dict):
    x_train_video = []
    y_train = []

    for path in json_paths:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for video in data:
            video_id = video["video_id"]
            if video_id not in video_features_dict:
                print(f"🚨 Video ID Eşleşmedi: {video_id}")
                continue

            video_features_data = video_features_dict[video_id]  # CLIP özelliklerini al
            tsum = video["tsum"]
            vsum = video["vsum"]

            if not tsum or not vsum:
                print(f"⚠️ Uyarı: Video {video_id} için caption veya segment yok!")
                continue

            for i, (caption, segment_group) in enumerate(zip(tsum, vsum)):
                caption_str = tokenizer.bos_token+" "+ re.sub(r'[^\w\s]', '', caption.lower()) + " "+tokenizer.eos_token

                if len(caption_str.split()) > MAX_TOKENS:
                    caption_str = " ".join(caption_str.split()[:MAX_TOKENS])

                while len(caption_str.split()) < MAX_TOKENS-1:
                    caption_str += " " + tokenizer.pad_token

                tokenized = tokenizer(caption_str, truncation=True, max_length=MAX_TOKENS, padding='max_length', return_tensors="np")

                for segment in segment_group:
                    start_frame = int(segment[0])
                    end_frame = min(int(segment[1]), len(video_features_data))

                    if start_frame >= len(video_features_data) or start_frame >= end_frame:
                        print(f"🚨 Geçersiz Frame Aralığı: {start_frame} - {end_frame} (Toplam: {len(video_features_data)})")
                        continue

                    segment_features = video_features_data[start_frame:end_frame]

                    # 📌 **Segment uzunluğu 15'ten az ise padding yap**
                    if segment_features.shape[0] < MAX_SEGMENT_FRAMES:
                        padding = np.zeros((MAX_SEGMENT_FRAMES - segment_features.shape[0], FEATURE_DIM), dtype=np.float32)
                        segment_features = np.vstack([segment_features, padding])

                    # 📌 **Segment uzunluğu 15'ten büyükse ilk 15 frame al**
                    if segment_features.shape[0] > MAX_SEGMENT_FRAMES:
                        segment_features = segment_features[:MAX_SEGMENT_FRAMES]

                    # 📌 Son boyut kontrolü
                    if segment_features.shape != (MAX_SEGMENT_FRAMES, FEATURE_DIM):
                        print(f"🚨 Hatalı Segment Boyutu: {segment_features.shape}, Beklenen: {(MAX_SEGMENT_FRAMES, FEATURE_DIM)}")
                        continue

                    x_train_video.append(segment_features)
                    y_train.append(tokenized["input_ids"][0])

    x_train_video = np.array(x_train_video, dtype=np.float32)
    y_train = np.array(y_train, dtype=np.int32)

    print(f"✅ Toplam Veri Sayısı: {len(x_train_video)}")
    return x_train_video, y_train

# 📌 CLIP Özellikleri Dosyaları
clip_feature_paths = [
    "/content/drive/MyDrive/vid_sum_act/clip_features_1.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_2.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_3.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_4.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_5.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_7.json"
]

# 📌 train_videoxum.json Dosyası
json_paths = ["/content/drive/MyDrive/vid_sum_act/train_videoxum.json"]

# 📌 CLIP Özelliklerini Yükleme
video_features_dict = load_clip_features(clip_feature_paths)

# 📌 Eğitim Verisini Hazırlama
x_train_video, y_train = prepare_training_data(json_paths, video_features_dict)
del video_features_dict

# 📌 Sonuçları Görüntüleme
print(f"x_train_video shape: {x_train_video.shape}")  # (örnek_sayısı, 15, 512)
print(f"y_train shape: {y_train.shape}")  # (örnek_sayısı, 128)



In [ ]:
import json
import numpy as np
import re

# 📌 Sabitler
MAX_TOKENS = 32  # Maksimum token sayısı
FEATURE_DIM = 512  # CLIP özellik boyutu
MAX_SEGMENT_FRAMES = 15  # Her segment için maksimum 15 frame


# 📌 CLIP Özelliklerini Yükleme (Validation için)
def load_clip_features(clip_feature_paths):
    video_features_dict = {}  # Video ID → Özellikler
    for path in clip_feature_paths:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
            for video_id, features in data.items():
                if isinstance(features, dict):
                    features = np.array(list(features.values()), dtype=np.float32)
                else:
                    features = np.array(features, dtype=np.float32)

                # 📌 Eğer iç içe liste varsa, ilk seviyeyle düzelt
                if len(features.shape) == 3:  # Örn: (1, N, 512) → (N, 512)
                    features = features[0]

                if len(features.shape) != 2 or features.shape[1] != FEATURE_DIM:
                    print(f"⚠️ Uyarı: {video_id} için geçersiz özellik boyutu: {features.shape}")
                    continue

                video_features_dict[f"v_{video_id}"] = features  # ID formatı düzeltildi!
    return video_features_dict

# 📌 Validation Verisini Hazırlama (Segment Bazlı Input)
def prepare_validation_data(json_paths, video_features_dict):
    x_val_video = []
    y_val = []
    yt_ids = []
    mismatch_count = 0
    for path in json_paths:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        for video in data:
            video_id = video["video_id"]
            if video_id not in video_features_dict:
                print(f"🚨 Video ID Eşleşmedi: {video_id}")
                continue

            video_features_data = video_features_dict[video_id]  # CLIP özelliklerini al
            tsum = video["tsum"]
            vsum = video["vsum"]

            if not tsum or not vsum:
                print(f"⚠️ Uyarı: Video {video_id} için caption veya segment yok!")
                continue

            for i, (caption, segment_group) in enumerate(zip(tsum, vsum)):
                caption_str = " " + re.sub(r'[^\w\s]', '', caption.lower()) + " "+tokenizer.eos_token

                if len(caption_str.split()) > MAX_TOKENS:
                    caption_str = " ".join(caption_str.split()[:MAX_TOKENS])

                while len(caption_str.split()) < MAX_TOKENS-1:
                    caption_str += " " + tokenizer.pad_token

                tokenized = tokenizer(caption_str, truncation=True, max_length=MAX_TOKENS, padding='max_length', return_tensors="np")

                for segment in segment_group:
                    start_frame = int(segment[0])
                    end_frame = min(int(segment[1]), len(video_features_data))

                    if start_frame >= len(video_features_data) or start_frame >= end_frame:
                        print(f"🚨 Geçersiz Frame Aralığı: {start_frame} - {end_frame} (Toplam: {len(video_features_data)})")
                        continue

                    segment_features = video_features_data[start_frame:end_frame]

                    # 📌 **Segment uzunluğu 15'ten az ise padding yap**
                    if segment_features.shape[0] < MAX_SEGMENT_FRAMES:
                        padding = np.zeros((MAX_SEGMENT_FRAMES - segment_features.shape[0], FEATURE_DIM), dtype=np.float32)
                        segment_features = np.vstack([segment_features, padding])

                    # 📌 **Segment uzunluğu 15'ten büyükse ilk 15 frame al**
                    if segment_features.shape[0] > MAX_SEGMENT_FRAMES:
                        segment_features = segment_features[:MAX_SEGMENT_FRAMES]

                    if segment_features.shape != (MAX_SEGMENT_FRAMES, FEATURE_DIM):
                        print(f"🚨 Hatalı Segment Boyutu: {segment_features.shape}, Beklenen: {(MAX_SEGMENT_FRAMES, FEATURE_DIM)}")
                        mismatch_count = mismatch_count+1
                        print(mismatch_count)
                        continue

                    x_val_video.append(segment_features)
                    y_val.append(tokenized["input_ids"][0])
                    yt_ids.append(video_id)
    x_val_video = np.array(x_val_video, dtype=np.float32)
    y_val = np.array(y_val, dtype=np.int32)

    print(f"✅ Toplam Validation Verisi: {len(x_val_video)}")
    return x_val_video, y_val,yt_ids

# 📌 Validation CLIP Özellikleri Dosyaları
clip_feature_paths_val = [
    "/content/drive/MyDrive/vid_sum_act/clip_features_val_1.json",
    "/content/drive/MyDrive/vid_sum_act/clip_features_val_2.json"
]

# 📌 Validation Caption Dosyası
json_paths_val = [
    "/content/drive/MyDrive/vid_sum_act/val_videoxum.json"
]

# 📌 Validation CLIP Özelliklerini Yükleme
video_features_dict_val = load_clip_features(clip_feature_paths_val)

# 📌 Validation Verisini Hazırlama
x_val_video, y_val,yt_ids = prepare_validation_data(json_paths_val, video_features_dict_val)
del video_features_dict_val

# 📌 Sonuçları Görüntüleme
print(f"x_val_video shape: {x_val_video.shape}")  # (örnek_sayısı, 15, 512)
print(f"y_val shape: {y_val.shape}")  # (örnek_sayısı, 128)


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=bdb46352ed872b4ea7eee2e9941d793cd2f3bc41b5116a6eb081a59fa3139e45
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
print(yt_ids[0])

v_IoiDAHNryTk


In [ ]:
# 📌 **Kaç tane test verisi ayıracağımızı belirliyoruz**
num_test_samples = 10000  # Son 500 veriyi test setine ayır

# 📌 **Test Setini Validation Setinden Ayırma**
x_test_video = x_val_video[-num_test_samples:]  # Son 500 veriyi al (Test seti)
y_test = y_val[-num_test_samples:]  # Son 500 caption'ı al (Test seti)
yt_ids = yt_ids[-num_test_samples:]
x_val_video = x_val_video[:-num_test_samples]  # Geri kalanları validation seti olarak bırak
y_val = y_val[:-num_test_samples]  # Geri kalan caption'ları validation olarak bırak

# 📌 **Sonuçları Yazdır**
print(f"✅ Yeni Validation Set Boyutu: {x_val_video.shape}")
print(f"✅ Yeni Test Set Boyutu: {x_test_video.shape}")


**VLM MODEL**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Config, GPT2LMHeadModel
#SELF ATTENTION'LI
# 📌 **Model Parametreleri**
MAX_TOKENS = 32
FEATURE_DIM = 512  # CLIP video embedding boyutu
NUM_FRAMES = 15
GPT2_EMBED_DIM = 768  # GPT-2'nin embedding boyutu
NUM_HEADS = 8  # Encoder için Multi-Head Attention
FF_DIM = 2048  # Feedforward Katmanı Boyutu
NUM_ENCODER_LAYERS = 6  # ⚡ Daha Derin Encoder
DROPOUT_RATE = 0.4  # 🔥 Dropout oranı (Overfitting önleme)

# 📌 **CLIP Özelliklerini Daha İyi Temsil Eden Feature Projection**
class FeatureProjection(nn.Module):
    def __init__(self, input_dim=FEATURE_DIM, output_dim=GPT2_EMBED_DIM):
        super(FeatureProjection, self).__init__()
        self.fc1 = nn.Linear(input_dim, input_dim * 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(input_dim * 2, output_dim)
        self.norm = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return self.norm(x)

# 📌 **Positional Encoding**
class PositionalEncoding(nn.Module):
    def __init__(self, seq_len, d_model):
        super(PositionalEncoding, self).__init__()
        angles = torch.arange(seq_len).unsqueeze(1) / torch.pow(10000, (2 * torch.arange(d_model).unsqueeze(0) // 2) / d_model)
        pos_enc = torch.zeros(seq_len, d_model)
        pos_enc[:, 0::2] = torch.sin(angles[:, 0::2])
        pos_enc[:, 1::2] = torch.cos(angles[:, 1::2])
        self.register_buffer('pos_encoding', pos_enc.unsqueeze(0))

    def forward(self, x):
        return x + self.pos_encoding.to(x.device)

# 📌 **Geliştirilmiş Transformer Encoder (Self-Attention Eklendi!)**
class TransformerEncoder(nn.Module):
    def __init__(self):
        super(TransformerEncoder, self).__init__()
        self.pos_encoding = PositionalEncoding(NUM_FRAMES, GPT2_EMBED_DIM)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=GPT2_EMBED_DIM,
            nhead=NUM_HEADS,
            dim_feedforward=FF_DIM * 2,
            dropout=DROPOUT_RATE,
            activation="gelu",
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=NUM_ENCODER_LAYERS)

        # 🔹 **Ek Self-Attention Katmanı**
        self.self_attn = nn.MultiheadAttention(embed_dim=GPT2_EMBED_DIM, num_heads=NUM_HEADS, batch_first=True)
        self.norm = nn.LayerNorm(GPT2_EMBED_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, x):
        x = self.pos_encoding(x)
        x = self.dropout(x)
        x = self.transformer_encoder(x)

        # 🔹 **Self-Attention Ekle**
        self_attn_output, _ = self.self_attn(x, x, x)
        x = self.norm(x + self.dropout(self_attn_output))

        return x

# 📌 **Attention Adapter (Self-Attention ve Cross-Attention Eklendi!)**
class AttentionAdapter(nn.Module):
    def __init__(self):
        super(AttentionAdapter, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=GPT2_EMBED_DIM, num_heads=8, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(embed_dim=GPT2_EMBED_DIM, num_heads=8, batch_first=True)

        # 🔹 **Ek Self-Attention Katmanı**
        self.self_attn = nn.MultiheadAttention(embed_dim=GPT2_EMBED_DIM, num_heads=8, batch_first=True)

        self.norm1 = nn.LayerNorm(GPT2_EMBED_DIM)
        self.norm2 = nn.LayerNorm(GPT2_EMBED_DIM)
        self.norm3 = nn.LayerNorm(GPT2_EMBED_DIM)
        self.dropout = nn.Dropout(DROPOUT_RATE)

    def forward(self, video_features):
        attn_output, _ = self.multihead_attn(video_features, video_features, video_features)
        x = self.norm1(video_features + self.dropout(attn_output))

        # 🔹 **Ek Self-Attention**
        self_attn_output, _ = self.self_attn(x, x, x)
        x = self.norm2(x + self.dropout(self_attn_output))

        # 🔹 **Cross-Attention**
        cross_attn_output, _ = self.cross_attn(x, x, x)
        return self.norm3(x + self.dropout(cross_attn_output))

# 📌 **Video Captioning Model (Özelleştirilmiş, Self-Attention Eklenmiş)**
class VideoCaptioningModel(nn.Module):
    def __init__(self):
        super(VideoCaptioningModel, self).__init__()
        self.feature_projection = FeatureProjection()
        self.encoder = TransformerEncoder()
        self.adapter = AttentionAdapter()

        config = GPT2Config.from_pretrained("gpt2", add_cross_attention=True)
        self.gpt2 = GPT2LMHeadModel.from_pretrained("gpt2", config=config)

        for param in self.gpt2.parameters():
            param.requires_grad = True

    def forward(self, video_features, captions):
        projected_features = self.feature_projection(video_features)
        encoder_output = self.encoder(projected_features)
        adapted_encoding = self.adapter(encoder_output)
        gpt2_output = self.gpt2(input_ids=captions, encoder_hidden_states=adapted_encoding, return_dict=True)
        return gpt2_output.logits

# 📌 **Modeli Oluştur**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VideoCaptioningModel().to(device)

# 📌 **Modelin Trainable Parametrelerini Kontrol Et**
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Fine-Tuning için Trainable Parametre Sayısı: {num_params}")


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm  # Progress bar için

# **Hyperparametreler**
BATCH_SIZE = 64
EPOCHS = 10
PATIENCE = 4  # 🔥 Early Stopping için
GRADIENT_CLIP = 1.0  # 🔥 Gradient Clipping için max norm

# **GPU kullanımı kontrol et**
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# **Optimizer ve Loss Fonksiyonu**
#optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)  # 🔥 Weight Decay eklendi
#loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.1)  # 🔥 Label Smoothing eklendi
import torch.optim as optim

# 🔥 GPT-2'nin parametrelerini daha düşük öğrenme oranıyla eğit
gpt2_params = []
new_params = []

for name, param in model.named_parameters():
    if "gpt2" in name:
        gpt2_params.append(param)  # 🔥 GPT-2'nin tüm katmanları düşük LR ile eğitilecek
    else:
        new_params.append(param)  # 🔥 Yeni eklenen Transformer Encoder daha yüksek LR ile eğitilecek

# 🔥 Farklı öğrenme oranları ile optimizer tanımla
optimizer = optim.AdamW([
    {"params": gpt2_params, "lr": 1e-6},  # ✅ **GPT-2'nin tüm katmanları düşük LR ile güncellenecek**
    {"params": new_params, "lr": 1e-5}   # ✅ **Yeni katmanlar hızlı öğrenmesi için büyük LR ile eğitilecek**
], weight_decay=0.01)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)  # 0 olan padding tokenlarını yok say

# **Dataları PyTorch Tensorlarına Çevir**
x_train_tensor = torch.tensor(x_train_video, dtype=torch.float32).to(device)
y_train_input = torch.tensor(y_train[:, :-1], dtype=torch.long).to(device)  # Decoder inputları
y_train_target = torch.tensor(y_train[:, 1:], dtype=torch.long).to(device)  # Decoder hedefleri

x_val_tensor = torch.tensor(x_val_video, dtype=torch.float32).to(device)
y_val_input = torch.tensor(y_val[:, :-1], dtype=torch.long).to(device)
y_val_target = torch.tensor(y_val[:, 1:], dtype=torch.long).to(device)

# **DataLoader ile Batch Haline Getir**
train_dataset = TensorDataset(x_train_tensor, y_train_input, y_train_target)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_input, y_val_target)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# **Early Stopping İçin Değişkenler**
best_loss = float("inf")
patience_counter = 0

# **Eğitim Fonksiyonu**
def train_model():
    global best_loss, patience_counter  # Early Stopping için değişkenleri dışarıdan al
    for epoch in range(EPOCHS):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=True)

        for x_batch, y_input_batch, y_target_batch in progress_bar:
            optimizer.zero_grad()
            output = model(x_batch, y_input_batch)
            loss = loss_fn(output.view(-1, output.size(-1)), y_target_batch.view(-1))

            # **Backward Pass**
            loss.backward()

            # 🔥 Gradient Clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=GRADIENT_CLIP)

            optimizer.step()

            total_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())  # Progress bar’a loss yazdır

        avg_train_loss = total_train_loss / len(train_loader)

        # **Validation**
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for x_batch, y_input_batch, y_target_batch in val_loader:
                output = model(x_batch, y_input_batch)
                loss = loss_fn(output.view(-1, output.size(-1)), y_target_batch.view(-1))
                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # **Early Stopping Kontrolü**
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_model.pth")  # En iyi modeli kaydet
            print("✅ En iyi model kaydedildi.")
        else:
            patience_counter += 1
            print(f"⏳ Early Stopping Patience: {patience_counter}/{PATIENCE}")
            if patience_counter >= PATIENCE:
                print("🛑 Early Stopping: En iyi model yüklendi.")
                #model.load_state_dict(torch.load("best_model.pth"))
                return  # Eğitim durduruldu

# **Eğitimi Başlat**
train_model()


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/vid_sum_act/bml-selfattn-gpt_torch_2.pth")  # En iyi modeli kaydet